In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
import glob
import traceback

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
        
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [8]:
scrapers = [state.split('.')[0] for state in glob.glob('*county.ipynb')]

In [9]:
len(scrapers)

15

In [10]:
import importlib

for scraper in scrapers:
    i = importlib.import_module(scraper)
    try:
        i.run()
    except:
        print(f'Error scraping {scraper}.')
        traceback.print_exc()

importing Jupyter notebook from NJ by county.ipynb
importing Jupyter notebook from CA by county.ipynb
importing Jupyter notebook from WA by county.ipynb
importing Jupyter notebook from TX by county.ipynb
importing Jupyter notebook from NY by county.ipynb
importing Jupyter notebook from AK by county.ipynb
importing Jupyter notebook from CO by county.ipynb
importing Jupyter notebook from MI by county.ipynb
importing Jupyter notebook from VA by county.ipynb
importing Jupyter notebook from FL by county.ipynb
importing Jupyter notebook from OH by county.ipynb
importing Jupyter notebook from MD by county.ipynb
importing Jupyter notebook from PA by county.ipynb
importing Jupyter notebook from TN by county.ipynb
Error scraping TN by county.
importing Jupyter notebook from WY by county.ipynb


Traceback (most recent call last):
  File "/Users/ckirby/.pyenv/versions/3.7.2/envs/ipython/lib/python3.7/site-packages/pandas/core/internals/construction.py", line 500, in _list_to_arrays
    content, columns, dtype=dtype, coerce_float=coerce_float
  File "/Users/ckirby/.pyenv/versions/3.7.2/envs/ipython/lib/python3.7/site-packages/pandas/core/internals/construction.py", line 583, in _convert_object_array
    "{con} columns".format(col=len(columns), con=len(content))
AssertionError: 2 columns passed, passed data had 1 columns

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-10-a6fb3440f6a4>", line 6, in <module>
    i.run()
  File "<string>", line 2, in run
  File "<string>", line 16, in fetch
  File "/Users/ckirby/.pyenv/versions/3.7.2/envs/ipython/lib/python3.7/site-packages/pandas/core/frame.py", line 450, in __init__
    arrays, columns = to_arrays(data, columns, dtype=dtype)
  File "/Users/ckirby/.pye

In [3]:
import HI_by_county as c

importing Jupyter notebook from HI by county.ipynb


In [4]:
c.run()